# Capstone: Rcommender System
## 1. Web Scraping and Data Collection

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
import numpy as np
import regex as re

import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,ElementNotVisibleException

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [304]:
#step 1 
#Get url for product in each category
driver = webdriver.Chrome('./chromedriver')

#Categories include facial cleanser, toner and moisturizer
productcat = ['face-wash-facial-cleanser', 'facial-toner-skin-toner', 'moisturizer-skincare']

#create a dataframe
df = pd.DataFrame(columns=['Category', 'URL'])

for cat in productcat:
    driver.get('https://www.sephora.com/shop/'+cat+'?pageSize=300')
    time.sleep(1)

    elem = driver.find_element_by_tag_name("body")
    
    #scroll page down to deal with lazy-load webpages
    no_of_pagedowns = 10
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1
    
    #find url
    pi = driver.find_elements_by_class_name("css-ix8km1")

    producturl = []
    for a in pi:
        subURL = a.get_attribute('href')
        producturl.append(subURL)
    
    dic = {'Category': cat, 'URL': producturl}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

In [305]:
df.groupby('Category').count()

,URL
Category,
face-wash-facial-cleanser,96
facial-toner-skin-toner,96
moisturizer-skincare,96


In [306]:
#clean product url 
newurl = []
for i in df.URL:
    newi = i.split('?')[0]
    newurl.append(newi)
df.URL = newurl

In [145]:
df.to_csv('../datasets/url.csv')

In [149]:
#step 2
#scrape product brands, names, ratings, prices, descriptions, ingredients and pictures
df_info = pd.DataFrame(columns=['brand', 'name', 'rating', 'price',
                                'descriptions', 'ingredients','pic','productsize'])
df = pd.concat([df, df_info], axis = 1)

In [ ]:
driver = webdriver.Chrome('./chromedriver')
for i in range(len(df)):
    driver.get(df.URL[i])
    time.sleep(5)

    try:
        driver.find_element_by_class_name('css-fslzaf').click()
    except ElementNotVisibleException:
        pass
    except NoSuchElementException:
        pass
    
    #product brand and name
    item = driver.find_element_by_class_name('css-a1jw00').text.split('\n')
    df.brand[i] = item[0]
    df.name[i] = item[1]
    
    #price
    df.price[i] = driver.find_element_by_class_name('css-14hdny6').text
    
    #descriptions
    df.descriptions[i]= driver.find_element_by_class_name('css-1rny024').text
    
    #one page down
    elem = driver.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    #ingredient
    #we need to click ingredient tab in product infomation table to get the text
    driver.find_element_by_id('tab2').click()
    df.ingredients[i] = driver.find_element_by_id('tabpanel2').text
    
    #scoll down so the website will show rating and review box
    no_of_pagedowns = 3
    while no_of_pagedowns:
        elem = driver.find_element_by_tag_name("body")
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        no_of_pagedowns-=1
    
    #rating 
    #there may be some product don't have ratings 
    try:
        ra = driver.find_element_by_class_name('css-1eqf5yr').text
        df.rating[i] = float(ra.split(' / ')[0])
    except NoSuchElementException:
        df.rating[i] = np.nan
    
    #pic
    driver.find_element_by_class_name('css-1juuxmf').click()
    time.sleep(1)
    df.pic[i] = driver.find_element_by_class_name('css-1glglyy').get_attribute('src')
    
    #productsize
    try:
        s = driver.find_element_by_class_name('css-12wl10d').text
        df.productsize[i] = s
    
    except NoSuchElementException:
        s = driver.find_element_by_class_name('css-1qf1va5').text
        df.productsize[i] = s
    
    time.sleep(0.5)
     

In [153]:
#save file
df.to_csv('../datasets/product_info_raw.csv')

In [3]:
df

,Category,URL,brand,name,rating,price,descriptions,ingredients,pic
0,face-wash-facial-cleanser,https://www.sephora.com/product/soy-face-clean...,FRESH,Soy Face Cleanser,4.4,$38.00,"What it is: A bestselling, gently powerful fac...",-Amino Acid-rich Soy Proteins: Help maintain s...,https://www.sephora.com/productimages/sku/s487...
1,face-wash-facial-cleanser,https://www.sephora.com/product/kale-spinach-g...,YOUTH TO THE PEOPLE,Superfood Antioxidant Cleanser,4.3,$36.00,What it is: A daily green juice cleanse for yo...,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...",https://www.sephora.com/productimages/sku/s186...
2,face-wash-facial-cleanser,https://www.sephora.com/product/green-clean-ma...,FARMACY,Green Clean Makeup Removing Cleansing Balm,4.6,$34.00,What it is: An award-winning makeup remover an...,-Sunflower and Ginger Root Oils: Melt even stu...,https://www.sephora.com/productimages/sku/s189...
3,face-wash-facial-cleanser,https://www.sephora.com/product/sugar-strawber...,FRESH,Sugar Strawberry Exfoliating Face Wash,4.5,$33.00,What it is: A gentle exfoliating face cleanser...,-Blend of White Sugar Crystals: Help smooth an...,https://www.sephora.com/productimages/sku/s222...
4,face-wash-facial-cleanser,https://www.sephora.com/product/purity-made-si...,PHILOSOPHY,Purity Made Simple Cleanser,4.5,$24.00,"What it is: A face wash for easy, one-step cle...",-Sodium Lauroamphoacetate: Acts as a mild surf...,https://www.sephora.com/productimages/sku/s407...
5,face-wash-facial-cleanser,https://www.sephora.com/product/soy-face-clean...,FRESH,Soy Face Cleanser Limited Edition,4.7,$42.00,What it is: A limited-edition 20th anniversary...,-Amino Acid-rich Soy Proteins: Help maintain t...,https://www.sephora.com/productimages/sku/s222...
6,face-wash-facial-cleanser,https://www.sephora.com/product/the-deep-clean...,TATCHA,The Deep Cleanse Exfoliating Cleanser,4.4,$38.00,Which skin type is it good for?\n✔ Normal\n✔ O...,"-Japanese Luffa fruit Exfoliant: Hydrates, cla...",https://www.sephora.com/productimages/sku/s202...
7,face-wash-facial-cleanser,https://www.sephora.com/product/beste-no-9-jel...,DRUNK ELEPHANT,Beste™ No. 9 Jelly Cleanser,3.8,$32.00,Which skin type is it good for?\n✔ Normal\n✔ O...,-Mild Surfactant Blend: A stack of mild cleans...,https://www.sephora.com/productimages/sku/s202...
8,face-wash-facial-cleanser,https://www.sephora.com/product/limited-editio...,YOUTH TO THE PEOPLE,Limited Edition Pride Superfood Antioxidant Cl...,4.9,$36.00,What it is: A daily green juice cleanse for yo...,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...",https://www.sephora.com/productimages/sku/s223...
9,face-wash-facial-cleanser,https://www.sephora.com/product/pure-one-step-...,TATCHA,Pure One Step Camellia Oil Cleanser,4.5,$48.00,Which skin type is it good for?\n✔ Normal\n✔ O...,-Japanese Camellia Oil (Tsubaki): Seals in moi...,https://www.sephora.com/productimages/sku/s167...


In [4]:
#step 3 
#scrap reviews for each product

In [95]:
#define click function to help keep clicking on the same button
def click(x):
    try:
        driver.find_element_by_class_name(x).click()
        time.sleep(1)
        click(x)
    except ElementNotVisibleException:
        driver.find_element_by_class_name('css-fslzaf').click()
        click(x)
    except NoSuchElementException:
        pass

#fill in NaN for missing informations
def fillin(rawlist,infolist):
    for i in range(len(rawlist)-1):
        if str(infolist[i]) in str(rawlist[i]):
            pass

        else:
            infolist.insert(i,np.nan)

    if len(infolist) < len(rawlist):
        infolist.append(np.nan)
            

In [ ]:
review = pd.DataFrame(columns=['Category','brand', 'name', 'user', 'stars',
                                'short_review', 'long_review','helpfulness','time'])
driver = webdriver.Chrome('./chromedriver')
for i in range(97,98):
    driver.get(df.URL[i])
    time.sleep(2)
    df_review = pd.DataFrame(columns=['Category','brand', 'name', 'user', 'stars',
                                'short_review', 'long_review','helpfulness','time'])
    
    try:
        driver.find_element_by_class_name('css-fslzaf').click()
    except ElementNotVisibleException:
        pass
    except NoSuchElementException:
        pass
    
    
    no_of_pagedowns = 5
    while no_of_pagedowns:
        elem = driver.find_element_by_tag_name("body")
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1
    
    start_time = time.time()
    click('css-1phfyoj')
    print(time.time()-start_time)
    
    #user profile
    up = [e.text for e in driver.find_elements_by_class_name('css-81z9n1')][1:-2]
    
    
    #user review
    ur = [e.text for e in driver.find_elements_by_class_name('css-12yc2vd')]
    
    #user name
    name =  [e.text for e in driver.find_elements_by_class_name('css-10n46hy')]
    #fill in missing user name with nan
    fillin(up,name)
    
    #get stars 
    stars = [int(e.get_attribute('aria-label')[0]) for e in driver.find_elements_by_class_name('css-5quttm')]
    
    #short review
    sr = [e.text for e in driver.find_elements_by_class_name('css-ai9pjd')]
    #fill in missing values with nan
    fillin(ur,sr)
    
    #long review
    lr = [e.text for e in driver.find_elements_by_class_name('css-1p4f59m')]
    
    #helpfulness 
    h = [e.text for e in driver.find_elements_by_class_name('css-39esqn')]
    helpfulness=[int(re.findall('(\d+)',e)[1]) for e in h]
    
    ###time posted
    #clean up xx d ago
    t = [e.text for e in driver.find_elements_by_class_name('css-1mfxbmj')]
    
    df_review.user = name
    df_review.stars = stars
    df_review.short_review = sr
    df_review.long_review = lr
    df_review.helpfulness = helpfulness
    df_review.time = t
    
    df_review.loc[:,'Category'] = df.Category[i]
    df_review.loc[:,'brand'] = df.brand[i]
    df_review.loc[:,'name'] = df.name[i]
    
    review = pd.concat([review,df_review])
    print('product',i,': ',time.time()-start_time)


In [4]:
review.to_csv('../datasets/review.csv')